In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(model="qwen2.5:3b")

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [5]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [8]:
quiz_generate_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

quiz_eval_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [9]:
quiz_chain = LLMChain(llm = llm, prompt=quiz_generate_prompt, output_key="quiz", verbose = True)

C:\Users\Yashodhan Jaltare\AppData\Local\Temp\ipykernel_9416\111205066.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm = llm, prompt=quiz_generate_prompt, output_key="quiz", verbose = True)


In [10]:
review_chain = LLMChain(llm = llm, prompt = quiz_generate_prompt, output_key="review", verbose=True)

In [11]:
generate_eval_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [15]:
file_path = r"data.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    TEXT = file.read()

In [16]:
NUMBER = 5
SUBJECT = "UML Designing"
TONE = "simple"

In [17]:
with get_openai_callback() as cb:
    response=generate_eval_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Yashodhan Jaltare\AppData\Local\Temp\ipykernel_9416\3752855207.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_eval_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:What is UML? Everything You Need to Know About Unified Modeling Language
Diagrams for Software EngineeringSoftware Development

What is UML?
UML, or Unified Modeling Language, is a visual modeling language that helps software developers visualize and construct new systems. 

It’s not a programming language — it’s a set of rules specifically for drawing diagrams. There are many types of software engineering diagrams, but think of this language like the software engineer’s version of a blueprint.

If you're brand new to UML, you might like our resource on UML Basics before digging in to UML diagrams. To learn more about UML diagramming, read on or jump ahead to one of the following sections:

Table of Contents
What is UML?
Video Intro to UML Diagramming
Common UML Notations
History of the Unified Modeling Language
What does a UML Diagram Do? Why Software Engineers Use the Unified M

In [18]:
quiz=response.get("quiz")

In [19]:
# Get json block from the response which is in between ```json and ``` 
quiz = quiz.replace("### RESPONSE_JSON","")
json_block = json.loads(quiz)

In [20]:
for question in json_block:
	print(json_block[question]["mcq"])
	print(json_block[question]["options"])
	print(json_block[question]["correct"])

What does the Unified Modeling Language (UML) primarily help software developers to do?
{'a': 'To create and maintain source code in a programming language', 'b': 'To visualize and construct new systems through diagrams', 'c': 'To debug running applications during development', 'd': 'To manage project schedules and deadlines'}
b
What is the main reason UML diagrams are used in software engineering?
{'a': 'To generate executable code from visual diagrams', 'b': 'To improve communication and understanding among team members', 'c': 'To replace traditional documentation methods entirely', 'd': 'To automate testing processes during development'}
b
Which of the following is NOT a type of UML diagram?
{'a': 'Activity Diagram', 'b': 'Sequence Diagram', 'c': 'Flowchart', 'd': 'Deployment Diagram'}
c
What is the purpose of UML packages?
{'a': 'To group related elements for better organization and clarity in diagrams', 'b': 'To define specific attributes for individual objects within a class', 'c

In [21]:
quiz_table_data = []
for key, value in json_block.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [23]:
quiz_table_data

[{'MCQ': 'What does the Unified Modeling Language (UML) primarily help software developers to do?',
  'Choices': 'a: To create and maintain source code in a programming language | b: To visualize and construct new systems through diagrams | c: To debug running applications during development | d: To manage project schedules and deadlines',
  'Correct': 'b'},
 {'MCQ': 'What is the main reason UML diagrams are used in software engineering?',
  'Choices': 'a: To generate executable code from visual diagrams | b: To improve communication and understanding among team members | c: To replace traditional documentation methods entirely | d: To automate testing processes during development',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a type of UML diagram?',
  'Choices': 'a: Activity Diagram | b: Sequence Diagram | c: Flowchart | d: Deployment Diagram',
  'Correct': 'c'},
 {'MCQ': 'What is the purpose of UML packages?',
  'Choices': 'a: To group related elements for better organ

In [24]:
quiz=pd.DataFrame(quiz_table_data)

In [25]:
quiz.to_csv("gen_ai.csv",index=False)

In [26]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'11_12_2024_18_34_18'